# Vector field plottings 

In [1]:
import numpy as np 
import plotter # my 3d plotting code with plotly
import matplotlib.pyplot as plt 

## Two Example Fields and Call Sequences

In [2]:
# Define grid
x1, y1, z1 = np.meshgrid(
    np.linspace(-1, 1, 5),
    np.linspace(-1, 1, 5),
    np.linspace(-1, 1, 5)
)

# Simple radial field example 
u1 = x1
v1 = y1
w1 = z1

plotter.plotter(x1, y1, z1, u1, v1, w1, title = 'Example Radial Field Plot')

In [3]:
x2, y2, z2 = np.meshgrid(np.arange(-0.8, 1, 0.2),
                      np.arange(-0.8, 1, 0.2),
                      np.arange(-0.8, 1, 0.8))

u2 = np.sin(np.pi * x2) * np.cos(np.pi * y2) * np.cos(np.pi * z2) 
v2 = -np.cos(np.pi * x2) * np.sin(np.pi * y2) * np.cos(np.pi * z2) 
w2 = (np.sqrt(2.0 / 3.0) * np.cos(np.pi * x2) * np.cos(np.pi * y2) * np.sin(np.pi * z2)) 

plotter.plotter(x2, y2, z2, u2, v2, w2, title = 'An example of a More complicated Field')

clear to me that some refinement needs doing on the plotting code so that we can see the underlying fields a lot clearer. This will be done in time

## Generating Physics Related Vector Fields

This is for the purpose of later having things to compare to. Possible examples of reasonable fields include:

- field around infinite 1d wire
- field around coil of wire
- field around magnetised sphere
- bar magnet? 

Plan is to make use of boundary conditions. If we have certain circumstances in which one field may be continuous but the other is not, then by allowing communication between these two can lead to accurate fields being derived for aggregate scenarios. 

In [4]:
from constants import CONSTANTS  # handy place to lump all of these and avoid typos 

In [5]:
def field_around_current_carrying_wire(I:float, 
                                       CONSTANTS:dict, 
                                       x_extent:list = [-0.5, 0.5], 
                                       y_extent:list = [-0.5, 0.5],
                                       z_extent:list = [-0.5, 0.5], 
                                       resolution:float = 10) -> list[np.ndarray]:
    '''
    A current carrying wire will have a magnetic field around it. The wire will run parallel to the z
    axis, and be placed at the origin of the x,y plane. Current will be travelling in the direction of positive z.

    Direction of the current is dependent on the direction of travel. When it comes time to compare these outputs 
    to that of the GP, we can crank up the resolution of this and see if it can recreate the field.
    
    Args:
        I (float): Current which the wire is carrying.
        CONSTANTS (dict): Dictionary of all of the physical constants which could be useful.

        x_extent (list[int]): extent in the x direction of the final field. In SI Units, this will be meters.
        y_extent (list[int]): extent in the y direction of the final field. In SI Units, this will be meters.
        z_extent (list[int]): extent in the z direction of the final field. In SI Units, this will be meters.

    Returns:
        6 numpy nd arrays 
    '''
    x, y, z = np.meshgrid(
        np.linspace(x_extent[0], x_extent[1], resolution),
        np.linspace(y_extent[0], y_extent[1], resolution),
        np.linspace(z_extent[0], z_extent[1], resolution)
    )

    r = np.sqrt(x**2 + y**2) + 1e-3  # avoid div by zero
    theta = np.arctan2(y, x)

    feild_strength = CONSTANTS["mu_naught"] * I / (2 * CONSTANTS["pi"])

    u = -feild_strength / r * np.sin(theta)
    v = feild_strength / r * np.cos(theta)
    w = np.zeros_like(z)

    return x, y, z, u, v, w

# call
x_wire, y_wire, z_wire, fields_x_wire, fields_y_wire, fields_z_wire = field_around_current_carrying_wire(100000000,CONSTANTS)
plotter.plotter(x_wire, y_wire, z_wire, fields_x_wire, fields_y_wire, fields_z_wire, title = 'B fields around current carrying wire')

In [6]:
def field_around_toroidal_current_carrying_wire(I:float, 
                                                ring_radius:float = 0.5,
                                                CONSTANTS:dict = CONSTANTS, 
                                                x_extent:list = [-0.5, 0.5], 
                                                y_extent:list = [-0.5, 0.5],
                                                z_extent:list = [-0.5,0.5], 
                                                resolution:float = 10, 
                                                integral_discretisation:float = 1000) -> list[np.ndarray]:
    '''
    In the case of a circulating current, a magnetic field is formed inside of it. This is a nice parity to the previous case. 
    We will place our coil in the centre of the x,y plane, with current circulating anticlockwise as you look at it down the z 
    axis. By the right hand rule, this will lead to a magnetic field which is aligned directly up the z axis. 
    
    Args:
        I (float): Current which the wire is carrying.
        ring_radius (float): Radius of our current carrying ring
        CONSTANTS (dict): Dictionary of all of the physical constants which could be useful.
        x_extent (list[int]): extent in the x direction of the final field. In SI Units, this will be meters.
        y_extent (list[int]): extent in the y direction of the final field. In SI Units, this will be meters.
        z_extent (list[int]): extent in the z direction of the final field. In SI Units, this will be meters.
        resolution (int): side length of rectangular grid
        integral_discretisation (int): how many segments we break up our integral into.

    Returns:
        6 numpy nd arrays 
    '''
    mu0, Pi = CONSTANTS['mu_naught'], CONSTANTS['pi']


    x, y, z = np.meshgrid(
        np.linspace(x_extent[0], x_extent[1], resolution),
        np.linspace(y_extent[0], y_extent[1], resolution),
        np.linspace(z_extent[0], z_extent[1], resolution)
    )
    Bx, By, Bz = np.zeros_like(x), np.zeros_like(y), np.zeros_like(z)


    # Biot savart prefactor
    prefactor = mu0 * I / (4*Pi)
    
    # Discretise the loop into large number of pieces for summation approximation 
    dphi = 2*np.pi / integral_discretisation
    phis = np.linspace(0, 2*Pi, integral_discretisation, endpoint=False)
    
    for phi in phis:
        # Coordinates of the source point on the loop
        xprime = ring_radius*np.cos(phi)
        yprime = ring_radius*np.sin(phi)
        zprime = 0.0
        
        dlx = -ring_radius * np.sin(phi) * dphi
        dly =  ring_radius * np.cos(phi) * dphi
        dlz =  0.0
        
        # r - r'
        rx = x - xprime
        ry = y - yprime
        rz = z - zprime
        
        # distance^3 (have to add small amount for numerical stability )
        r3 = (rx**2 + ry**2 + rz**2)**1.5 + 0.01  
        
        # cross product dl' x (r-r'):
        cross_x = dly*rz - dlz*ry  # = R cos(phi)*z * dphi
        cross_y = dlz*rx - dlx*rz  # = R sin(phi)*z * dphi
        cross_z = dlx*ry - dly*rx  
       
        # Accumulate in B
        Bx += prefactor * cross_x / r3
        By += prefactor * cross_y / r3
        Bz += prefactor * cross_z / r3

    return x, y, z, Bx, By, Bz
# call
x_coil, y_coil, z_coil, fields_x_coil, fields_y_coil, fields_z_coil = field_around_toroidal_current_carrying_wire(100000000)
plotter.plotter(x_coil, y_coil, z_coil, fields_x_coil, fields_y_coil, fields_z_coil, title = 'B fields around current carrying loop') 

In [7]:
def field_around_infinite_solenoid(I:float, 
                                   CONSTANTS:dict, 
                                   x_extent:list = [-5, 5], 
                                   y_extent:list = [-5, 5],
                                   z_extent:list = [5,-5], 
                                   resolution:float = 10) -> list[np.ndarray]:
    '''
    To extend the previous case, we can also model an infinite current carrying solenoid, parallel to the z axis. 
    This is also a case for which the analytical solution is known exactly and hence offers a nice playground to 
    test the GP methods. 

    Will be interesting for investigating the tail behaviour of the GPs fitted field. 

    Args:
        I (float): Current which the wire is carrying.
        CONSTANTS (dict): Dictionary of all of the physical constants which could be useful.

        x_extent (list[int]): extent in the x direction of the final field. In SI Units, this will be meters.
        y_extent (list[int]): extent in the y direction of the final field. In SI Units, this will be meters.
        z_extent (list[int]): extent in the z direction of the final field. In SI Units, this will be meters.

    Returns:
        6 numpy nd arrays 
    '''
    pass

## Example fields from Kernels

In [8]:
import numpy as np 
import sys
sys.path.append('..')  
from GP_Implementation.Kernel import sample_vector_field

In [9]:
# Define grid
grid_x, grid_y, grid_z = np.meshgrid(
    np.linspace(-0.5, 0.5, 10),
    np.linspace(-0.5, 0.5, 10),
    np.linspace(-0.5, 0.5, 10)
)

In [10]:
grid_x, grid_y, grid_z, U, V, W = sample_vector_field(grid_x=grid_x, grid_y=grid_y, grid_z=grid_z,
                                                      kernel_func = 'divergence_free_kernel')
plotter.plotter(grid_x, grid_y, grid_z, U, V, W, title = 'Sample from a divergence free prior')

In [11]:
grid_x, grid_y, grid_z, U1, V1, W1 = sample_vector_field(grid_x=grid_x, grid_y=grid_y, grid_z=grid_z,
                                                         kernel_func = 'curl_free_kernel')
plotter.plotter(grid_x, grid_y, grid_z, U1, V1, W1, title = 'Sample from a curl free prior')

## Updates given measurements 

This is a 2 step process. To start, we make a function which can take in all of these vector fields, and return some selection of points. Following this sampling, we will feed this to our GP and see what happens.


### Generating samples

In [12]:
def sample_field(x, y, z, Bx, By, Bz, n_samples: int, random_seed: int = 0):
    '''
    Helper function to use to get random points from earlier fields. Samples n points from the given field data. 

    Args:
        x, y, z (np.ndarray): Meshgrid coordinates.
        Bx, By, Bz (np.ndarray): Magnetic field components.
        n_samples (int): Number of points to sample.

    Returns:
        Tuple of sampled coordinates and field components.
    '''
    np.random.seed(random_seed)
    
    resolution = x.shape[0]
    indices = np.random.choice(resolution * resolution * resolution, size=n_samples, replace=False)
    
    x_samples = x.flatten()[indices]
    y_samples = y.flatten()[indices]
    z_samples = z.flatten()[indices]
    Bx_samples = Bx.flatten()[indices]
    By_samples = By.flatten()[indices]
    Bz_samples = Bz.flatten()[indices]
    
    return x_samples, y_samples, z_samples, Bx_samples, By_samples, Bz_samples

In [13]:
x_samples_wire, y_samples_wire, z_samples_wire, Bx_samples_wire, By_samples_wire, Bz_samples_wire = sample_field(
    x_wire, y_wire, z_wire, fields_x_wire, fields_y_wire, fields_z_wire, n_samples=100
)
plotter.plotter(x_samples_wire, y_samples_wire, z_samples_wire, Bx_samples_wire, By_samples_wire, Bz_samples_wire, 
                title = 'Points given to our GP Regressor for current carrying wire')

In [14]:
x_samples_coil, y_samples_coil, z_samples_coil, Bx_samples_coil, By_samples_coil, Bz_samples_coil = sample_field(
    x_coil, y_coil, z_coil, fields_x_coil, fields_y_coil, fields_z_coil, n_samples=100
)
plotter.plotter(x_samples_coil, y_samples_coil, z_samples_coil, Bx_samples_coil, By_samples_coil, Bz_samples_coil, 
                title = 'Points given to our GP Regressor for current carrying loop')

### Performing Updates

Next, we plug these points into the updater and see what happens 

In [15]:
from GP_Implementation.Kernel import updated_vector_field

#### Divergence free

In [16]:
outputs_wire_div_free= updated_vector_field(
    x_samples_wire, y_samples_wire, z_samples_wire, 
    Bx_samples_wire, By_samples_wire, Bz_samples_wire, 
    x_wire, y_wire, z_wire, 
    kernel_func='divergence_free_kernel', 
    sigma_f =  0.1, l = 1.2323)

# unpack outputs 
updated_x_samples_wire_div_free =  outputs_wire_div_free[0]
updated_y_samples_wire_div_free =  outputs_wire_div_free[1]
updated_z_samples_wire_div_free =  outputs_wire_div_free[2]
updated_Bx_samples_wire_div_free = outputs_wire_div_free[3]
updated_By_samples_wire_div_free = outputs_wire_div_free[4]
updated_Bz_samples_wire_div_free = outputs_wire_div_free[5]

# from giving the code a large number of samples, the optimum 
# parameters seem to be about: sigma_f = 0.1, l = 1.2323 (this is from mle and it works poorly)
# sigma_f = 0.005, l = 0.8 seem to work okay 

Parameters for sigma_f and l specified, using these.


In [17]:
given_samples_wire = x_samples_wire.shape[0]
plotter.plotter(updated_x_samples_wire_div_free, updated_y_samples_wire_div_free, updated_z_samples_wire_div_free, 
                updated_Bx_samples_wire_div_free, updated_By_samples_wire_div_free, updated_Bz_samples_wire_div_free, 
                title = f'Output from GP Regression for n = {given_samples_wire} input vectors for current carrying wire')

In [18]:
outputs_coil_div_free = updated_vector_field(
    x_samples_coil, y_samples_coil, z_samples_coil, 
    Bx_samples_coil, By_samples_coil, Bz_samples_coil, 
    x_coil, y_coil, z_coil, 
    kernel_func='divergence_free_kernel', 
    sigma_f = 0.1, l = 0.1191608)

# unpack outputs
updated_x_samples_coil_div_free  = outputs_coil_div_free[0]
updated_y_samples_coil_div_free  = outputs_coil_div_free[1]
updated_z_samples_coil_div_free  = outputs_coil_div_free[2]
updated_Bx_samples_coil_div_free = outputs_coil_div_free[3]
updated_By_samples_coil_div_free = outputs_coil_div_free[4]
updated_Bz_samples_coil_div_free = outputs_coil_div_free[5]

# from giving the code a large number of samples, the optimum 
# parameters seem to be about: sigma_f = 0.1, l = 0.1191608
# sigma_f = 0.01, l = 1.5 seem to work okay 

Parameters for sigma_f and l specified, using these.


In [19]:
given_samples_coil = x_samples_coil.shape[0]

plotter.plotter(updated_x_samples_coil_div_free, updated_y_samples_coil_div_free, updated_z_samples_coil_div_free, 
                updated_Bx_samples_coil_div_free, updated_By_samples_coil_div_free, updated_Bz_samples_coil_div_free, 
                title = f'Output from GP Regression for n = {given_samples_coil} input vectors for current carrying wire')

#### Curl Free (most of these work very poorly)

In [20]:
outputs_wire_curl_free = updated_vector_field(
    x_samples_wire, y_samples_wire, z_samples_wire, 
    Bx_samples_wire, By_samples_wire, Bz_samples_wire, 
    x_wire, y_wire, z_wire, 
    kernel_func='curl_free_kernel', 
    sigma_f = 0.1, l = 0.3)

# unpack outputs 
updated_x_samples_wire_curl_free =  outputs_wire_curl_free[0]
updated_y_samples_wire_curl_free =  outputs_wire_curl_free[1]
updated_z_samples_wire_curl_free =  outputs_wire_curl_free[2]
updated_Bx_samples_wire_curl_free = outputs_wire_curl_free[3]
updated_By_samples_wire_curl_free = outputs_wire_curl_free[4]
updated_Bz_samples_wire_curl_free = outputs_wire_curl_free[5]

Parameters for sigma_f and l specified, using these.


In [21]:
plotter.plotter(updated_x_samples_wire_curl_free, updated_y_samples_wire_curl_free, updated_z_samples_wire_curl_free, 
                updated_Bx_samples_wire_curl_free, updated_By_samples_wire_curl_free, updated_Bz_samples_wire_curl_free)

In [ ]:
outputs_coil_curl_free= updated_vector_field(
    x_samples_coil, y_samples_coil, z_samples_coil, 
    Bx_samples_coil, By_samples_coil, Bz_samples_coil, 
    x_coil, y_coil, z_coil, 
    kernel_func='curl_free_kernel',
    sigma_f = 0.1, l = 0.3)

# unpack outputs 
updated_x_samples_coil_curl_free =  outputs_coil_curl_free[0]
updated_y_samples_coil_curl_free =  outputs_coil_curl_free[1]
updated_z_samples_coil_curl_free =  outputs_coil_curl_free[2]
updated_Bx_samples_coil_curl_free = outputs_coil_curl_free[3]
updated_By_samples_coil_curl_free = outputs_coil_curl_free[4]
updated_Bz_samples_coil_curl_free = outputs_coil_curl_free[5]

Parameters for sigma_f and l specified, using these.


In [23]:
plotter.plotter(updated_x_samples_coil_curl_free, updated_y_samples_coil_curl_free, updated_z_samples_coil_curl_free, 
                updated_Bx_samples_coil_curl_free, updated_By_samples_coil_curl_free, updated_Bz_samples_coil_curl_free)